# Regression (Notebook for Databricks)

We'll do the following steps to build our model, 

Steps:
1. Use the features: **`bedrooms`**, **`bathrooms`**, **`bathrooms_na`**, **`minimum_nights`**, and **`number_of_reviews`** as input to your VectorAssembler.
2. Build a Linear Regression Model
3. Evaluate the **`RMSE`** and the **`R2`**.

In [1]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession

# from pyspark import SparkConf, SparkContext
from datetime import datetime, date, timedelta
from dateutil import relativedelta
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.functions import to_timestamp, to_date
from pyspark.sql import functions as F
from pyspark.sql.functions import collect_list, collect_set, concat, first, array_distinct, col, size, expr
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Start the spark session (Although it is not required if notebook directly ran in Databricks)
spark = SparkSession.builder \
    .appName("Flight Data Analysis in Spark") \
    .getOrCreate()

24/05/01 17:53:00 WARN Utils: Your hostname, Rishikesans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.26 instead (on interface en0)
24/05/01 17:53:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/01 17:53:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/01 17:53:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Load Dataset and Train Model

In [3]:
file_path = "./cleaned_listings.csv"
# airbnb_df = spark.read.format("delta").load(file_path)

#Read the cleaned csv file 
airbnb_df = spark.read.csv(file_path, header="true", inferSchema="true", multiLine="true", escape='"')
train_df, test_df = airbnb_df.randomSplit([.8, .2], seed=42)

#Vectorize the dependent variables 

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression

vec_assembler = VectorAssembler(inputCols=["bedrooms","bathrooms","bathrooms_na","minimum_nights","number_of_reviews"],outputCol="features")

vtrain_df = vec_assembler.transform(train_df)
vtest_df = vec_assembler.transform(test_df)

lr_model = LinearRegression(labelCol="price").fit(vtrain_df)

In [7]:
pred_df = lr_model.transform(vtest_df)

regression_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="rmse")
rmse = regression_evaluator.evaluate(pred_df)
r2 = regression_evaluator.setMetricName("r2").evaluate(pred_df)

print(f"RMSE is {rmse}")
print(f"R2 is {r2}")

RMSE is 419.0121578376188
R2 is 0.07456171275982248


In [8]:
# OOS R-squared is low. Note this notebook is only for practice for Spark ML. 

In [6]:
for col, coef in zip(vec_assembler.getInputCols(), lr_model.coefficients):
    print(col, coef)
  
print(f"intercept: {lr_model.intercept}")

bedrooms 114.17777449189113
bathrooms -5.8636569331674835
bathrooms_na -93.46199646696445
minimum_nights 0.11479885115899408
number_of_reviews -0.2841304691298576
intercept: 89.84420157032639


## Additional notes on Spark Distributed Computing

## Distributed Setting

Although we can quickly solve for the parameters when the data is small, the closed form solution doesn't scale well to large datasets. 

Spark uses the following approach to solve a linear regression problem:

* First, Spark tries to use matrix decomposition to solve the linear regression problem. 
* If it fails, Spark then uses <a href="https://spark.apache.org/docs/latest/ml-advanced.html#limited-memory-bfgs-l-bfgs" target="_blank">L-BFGS</a> to solve for the parameters. L-BFGS is a limited-memory version of BFGS that is particularly suited to problems with very large numbers of variables. The <a href="https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm" target="_blank">BFGS</a> method belongs to <a href="https://en.wikipedia.org/wiki/Quasi-Newton_method" target="_blank">quasi-Newton methods</a>, which are used to either find zeroes or local maxima and minima of functions iteratively. 
